<a href="https://colab.research.google.com/github/shimjaeman/PyTorch-Tutorials/blob/main/07_(Pytorch)_Transfer_Learning_and_Fine_Tuning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Transfer Learning

## 1. Pre-Training

> Pre-training
> 
> 
> **사전 학습 모델**이란 기존에 자비어(Xavier) 등 임의의 값으로 초기화하던 모델의 가중치들을    **다른 문제(task)에 학습시킨 가중치들로 초기화**하는 방법이다.
> 

예를 들어, 텍스트 유사도 예측 모델을 만들기 전 감정 분석 문제를 **학습한 모델의 가중치를 활용해** 텍스트 유사도 **모델의 가중치로 활용**하는 방법이다.

즉, 감정 분석 문제를 학습하면서 얻은 **언어에 대한 이해를 학습**한 후 **그 정보를** 유사도 문제를 **학습하는 데 활용**하는 방식이다.

이때 사전 학습한 가중치를 활용해 **학습하고자 하는 본 문제**를 **하위 문제(downstream task)** 라 한다. 앞 예시에서 사전 학습한 모델인 감정 분석 문제가 **사전 학습 문제(pre-train task)** 가 된다.

## 2. Fine-Tuning

> fine-tuning
> 
> 
> **사전 학습한 모든 가중치 downstream task를 위한 최소한의 가중치 추가로 학습(미세 조정)**
> 

앞 예시의 사전 학습 방법인 감정 분석 문제에 사전 학습시킨 가중치와 더불어 텍스트 유사도를 위한 부가적인 가중치를 추가해 텍스트 유사도 문제를 학습하는 것이 미세 조정 방법이다.

## 3. Transfer Learning

> transfer learning                                                                                                                           이러한 사전 학습-파인튜닝을 하는 학습 과정을 전이학습(transfer learning)이라고 한다.          전이학습은 이미 잘 훈련된 모델이 있고, 해당 모델과 유사한 문제를 해결하는데 효율적으로   사용된다.
> 

딥러닝 사전 학습은 딥러닝에서 **weight와 Bias를 잘 초기화 시키는 방법**이다. 이러한 사전 학습을 통해서 효과적으로 층(layer)을 쌓아서 **여러 개의 은닉층(hidden layer)를 효율적으로 훈련**할 수 있다.

또한 사전 학습은 레이블된 학습 데이터가 필요하지 않아서 **비지도 학습(unsupervised learning)이 가능**하기 때문에 레이블 되지 않은 빅 데이터를 넣어 훈련시킬 수 있다는 장점이 있다.

사전학습 만으로 원하는 태스크를 수행하는 인공지능 엔진이 완성이 되지는 않고 사전 학습에 기반한 **Fine Tuning** 이 필요하다.이는 사전 학습 모델을 기반으로 새로운 목적(질의응답, 번역 등)을 위해 이미 학습된 weight나 bias를 미세하게 조정하는 과정이다.

-> **사전학습 모델에 추가 데이터 투입해 weight 업데이트**

## 참고 사이트

https://inhovation97.tistory.com/31

https://blog.naver.com/ssj860520/222853939422

# Transfer Learning modeling

In [ ]:
# 기본 라이브러리
import numpy as np
import pandas as pd
import sys

# Pytorch 라이브러리
import torch
import torchvision
import torch.nn as nn # SGD, Adam 등과 같은 optimizer
import torch.optim as optim # 모든 neural network 모듈
import torch.nn.functional as F # Parameterless functions
from torch.utils.data import DataLoader # 미니배치를 생성하여 쉽게 데이터 셋 처리
import torchvision.datasets as datasets 
import torchvision.transforms as transforms # augumetation을 위해 데이터 세트에서 수행할 수 있는 변환
from tqdm import tqdm  # progress bar (진행상황 확인)

In [ ]:
# 파이썬 코드를 재실행해도 같은 결과가 나오도록 랜덤 시드(random seed) 설정
torch.manual_seed(1)

In [ ]:
# Set device (Cuda / cpu)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
# Hyperparameters
num_classes = 10
learning_rate = 3e-4
batch_size = 64
num_epochs = 3

## finetuning

In [ ]:
# vgg 구조 확인
model = torchvision.models.vgg16(weights=True)
model.to(device)

/usr/local/lib/python3.8/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG16_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/vgg16-397923af.pth" to /root/.cache/torch/hub/checkpoints/vgg16-397923af.pth


  0%|          | 0.00/528M [00:00<?, ?B/s]

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1

In [ ]:
# fine tunnig Class
class Identity(nn.Module):
    def __init__(self):
        super(Identity, self).__init__()

    def forward(self, x):
      return x
model = torchvision.models.vgg16(weights="DEFAULT").to(device)
model.avgpool = Identity()
model.classifier = nn.Sequential(nn.Linear(512, num_classes))
model.to(device)

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1

In [ ]:
# If you want to do finetuning then set requires_grad = False
# Remove these two lines if you want to train entire model,
# and only want to load the pretrain weights.
for param in model.parameters():
    param.requires_grad = False

model = torchvision.models.vgg16(weights="DEFAULT").to(device)
model.avgpool = nn.Identity()
model.classifier = nn.Sequential(nn.Linear(512, 100), nn.ReLU(inplace=True), nn.Linear(100, num_classes))
model.to(device)

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1

## Result

In [ ]:
# Load Data
train_dataset = datasets.CIFAR10(root="dataset/", train=True, transform=transforms.ToTensor(), download=True)
train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)

  0%|          | 0/170498071 [00:00<?, ?it/s]

Extracting dataset/cifar-10-python.tar.gz to dataset/


In [ ]:
# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

In [ ]:
# Train Network
for epoch in range(num_epochs):
    losses = []
    for batch_idx, (data, targets) in enumerate(train_loader):
        # Get data to cuda if possible
        data = data.to(device=device)
        targets = targets.to(device=device)

        # Forward
        scores = model(data)
        loss = criterion(scores, targets)
        losses.append(loss.item())

        # Backward
        optimizer.zero_grad()
        loss.backward()

        # Gradient descent or adam step
        optimizer.step()
    print(f"Cost at epoch {epoch} is {sum(losses)/len(losses):.5f}")

100%|██████████| 782/782 [00:33<00:00, 23.01it/s]


Cost at epoch 0 is 0.73244


100%|██████████| 782/782 [00:27<00:00, 28.22it/s]


Cost at epoch 1 is 0.38412


100%|██████████| 782/782 [00:28<00:00, 27.21it/s]

Cost at epoch 2 is 0.25713


In [ ]:
# Check accuracy on training & test 
def check_accuracy(loader, model):
    num_correct = 0
    num_samples = 0
    model.eval() # eval mode에서 사용할 것이라고 모든 레이어에 선언하는 것

    # torch.no_grad() : 검증단계에서는 backprop을 하지 않을 것이기 때문에 
    # no_grad()로 감싸주고 이는 이 블럭 안에서는 그래디언트 트래킹을 안해도 된다 라고 선언 
    # ===> 메모리& 시간 감소
    with torch.no_grad():
        for x, y in tqdm(loader):

            # Move data to device
            x = x.to(device=device)
            y = y.to(device=device)

            # Forward pass
            scores = model(x)
            _, predictions = scores.max(1)

            # Check how many we got correct
            num_correct += (predictions == y).sum()

            # Keep track of number of samples
            num_samples += predictions.size(0)

    model.train() # eval => train 
    return num_correct / num_samples

In [ ]:
# Check accuracy on training & test to see how good our model
print(f"Accuracy on training set : {check_accuracy(train_loader, model)*100:.2f}")

100%|██████████| 782/782 [00:07<00:00, 100.61it/s]

Accuracy on training set : 89.22
